In [1]:
# Importamos las librerias

import os # Importamos el módulo os para interactuar con el sistema operativo.
import requests # Importamos el módulo requests para realizar solicitudes HTTP.
import sqlite3 # Importamos el módulo sqlite3 para trabajar con bases de datos SQLite.
import pandas as pd # Importamos pandas bajo el alias pd para manipulación y análisis de datos.

In [2]:
# Directorio de los archivos de datos sin procesar.
_data_root = 'data/data_mod/'

# Ubicación del almacenamiento de metadatos del pipeline.
_pipeline_root = './pipeline/'


In [3]:
# Importamos pandas bajo el alias pd para manipulación y análisis de datos.
import pandas as pd

# Importamos TensorFlow y sus componentes de TFX para la construcción de pipelines de machine learning.
import tensorflow as tf
import tfx
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.v1.components import ImportSchemaGen
import tensorflow_data_validation as tfdv # Importamos TensorFlow Data Validation (TFDV) para la validación de datos.

# Importamos clases y funciones adicionales necesarias para el contexto interactivo de TFX.
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

# Importamos pprint para imprimir objetos de manera más legible.
import pprint
pp = pprint.PrettyPrinter()

2024-03-04 05:45:18.619921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 05:45:19.869282: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-04 05:45:19.869502: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [4]:
# Configuracion de contexto interactivo
context = InteractiveContext(pipeline_root=_pipeline_root)

### Examplex gen

In [5]:
# Instanciamos CsvExampleGen con el conjunto de datos CSV de entrada.
example_gen = CsvExampleGen(input_base=_data_root)

In [6]:
# Ejecutamos el componente CsvExampleGen en el contexto actual.
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 82
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Estadisticas

In [ ]:
# Instanciamos StatisticsGen con el dataset ingestado por ExampleGen.
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Ejecutamos el componente StatisticsGen en el contexto actual.
context.run(statistics_gen)

In [ ]:
# Mostramos las estadísticas de salida generadas por StatisticsGen.
context.show(statistics_gen.outputs['statistics'])

### Inferir el esquema

In [ ]:
# Instanciamos SchemaGen con el dataset ingestado por StatisticsGen.
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Ejecutamos el componente SchemaGen en el contexto actual.
context.run(schema_gen)

In [ ]:
# Visualizamos el esquema generado por SchemaGen.
context.show(schema_gen.outputs['schema'])

### Curando el esquema

In [ ]:
# Importamos el módulo schema_pb2 del paquete tensorflow_metadata.proto.v0 para trabajar con el esquema.
from tensorflow_metadata.proto.v0 import schema_pb2

# Intentamos obtener la URI del esquema generado por SchemaGen.
try:
    # Get the schema uri
    schema_uri = schema_gen.outputs['schema']._artifacts[0].uri
# En caso de que context.run() no se haya ejecutado, imprimimos un mensaje indicando que no se pudo ejecutar.
except IndexError:
    print("context.run() was no-op")
    
    # Accedemos al directorio donde se encuentra el esquema.
    schema_path = './pipeline/SchemaGen/schema'
    dir_id = os.listdir(schema_path)[0]
    schema_uri = f'{schema_path}/{dir_id}'
    
# Cargamos el esquema desde el archivo schema.pbtxt en la URI proporcionada.
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

In [ ]:
# Modificando Hillshade_9am de 0 a 255
tfdv.set_domain(schema, "Hillshade_9am", schema_pb2.IntDomain(name = "Hillshade_9am", min = 0 , max = 255 ))

# Modificando Hillshade_Noon de 0 a 255
tfdv.set_domain(schema, "Hillshade_Noon", schema_pb2.IntDomain(name = "Hillshade_Noon", min = 0 , max = 255 ))

# Modificando Slope de 0 a 90
tfdv.set_domain(schema, "Slope", schema_pb2.IntDomain(name = "Slope", min = 0 , max = 90 ))

# Modificando Cover_Type de 0 a 6
tfdv.set_domain(schema,'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))

In [ ]:
# Mostramos el esquema utilizando la función display_schema de TensorFlow Data Validation (TFDV).

tfdv.display_schema(schema=schema)

### Entorno de Esquemas

In [ ]:
# Cargamos el conjunto de datos de servicio desde el archivo CSV.
serv_data_df = pd.read_csv("data/data_mod/datos_finales.csv")

# Seleccionamos una muestra aleatoria del 30% del conjunto de datos, utilizando una semilla aleatoria de 42 para reproducibilidad.
serv_data_df = serv_data_df.sample(frac = 0.30, random_state=42) 

# Eliminamos la columna 'Cover_Type' del conjunto de datos de servicio.
serv_data_df = serv_data_df.drop(columns=["Cover_Type"])

# Directorio para guardar los datos de servicio
SERVING_DIR = 'data/serving'
os.makedirs(SERVING_DIR, exist_ok=True)

# Guardar los datos de servicio en un archivo CSV
SERVING_DATA_CSV = os.path.join(SERVING_DIR, 'serving_dataset.csv')
serv_data_df.to_csv(SERVING_DATA_CSV, index=False)

# Definir opciones de estadísticas
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

# Calcular estadísticas del conjunto de datos de servicio
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA_CSV, stats_options=stats_options)

# Detectar anomalías en el conjunto de datos de servicio
anomalies = tfdv.validate_statistics(serving_stats, schema=schema)

# Mostrar las anomalías detectadas
tfdv.display_anomalies(anomalies)

In [ ]:
schema.default_environment.append('TRAINING')

# Agregamos un nuevo entorno de esquema predeterminado llamado 'SERVING'.
schema.default_environment.append('SERVING')

# Eliminamos la característica 'Cover_Type' del entorno 'SERVING' utilizando TFDV.
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')

In [ ]:
# Validamos las estadísticas del conjunto de datos de servicio en el entorno 'SERVING' utilizando el esquema modificado.
anomalies = tfdv.validate_statistics(serving_stats, schema=schema, environment='SERVING')

# Mostramos las anomalías detectadas en el conjunto de datos de servicio en el entorno 'SERVING'.
tfdv.display_anomalies(anomalies)

In [ ]:
# Declaramos la ruta al directorio del esquema actualizado.
UPDATED_SCHEMA_DIR = './pipeline/updated_schema'

# Creamos el directorio si no existe.
!mkdir -p {UPDATED_SCHEMA_DIR}

# Declaramos la ruta al archivo del esquema.
schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

# Guardamos el esquema modificado en el archivo especificado.
tfdv.write_schema_text(schema, schema_file)


In [ ]:
# Cargamos el esquema desde el directorio que acabamos de crear.
new_schema = tfdv.load_schema_text(schema_file)

# Mostramos visualmente el esquema cargado.
tfdv.display_schema(schema=new_schema)

### Nuevas Estadisticas

In [ ]:
# Definimos la ruta al archivo de esquema para la generación de esquemas importados.
imported_schema = ImportSchemaGen(schema_file=schema_file)

In [ ]:
# Ejecutamos el componente ImportSchemaGen.
context.run(imported_schema, enable_cache=False)

# Mostramos el esquema importado generado por el componente.
context.show(imported_schema.outputs['schema'])

In [ ]:
# Utilizamos StatisticsGen para calcular las estadísticas utilizando el esquema curado.
statistics_gen_updated = StatisticsGen(examples=example_gen.outputs['examples'], 
                                       stats_options=tfdv.StatsOptions(infer_type_from_schema=True),
                                      schema=imported_schema.outputs['schema'])
    

# Ejecutamos el componente StatisticsGen.
context.run(statistics_gen_updated)

In [ ]:
# Mostramos las estadísticas generadas por el componente StatisticsGen utilizando el esquema curado.
context.show(statistics_gen_updated.outputs['statistics'])

### Comprobar anomalias

In [ ]:
# Creamos el componente ExampleValidator con las estadísticas y el esquema curado.
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=imported_schema.outputs['schema'])
    
# Ejecutamos el componente ExampleValidator.
context.run(example_validator)

In [ ]:
# Mostramos las anomalías detectadas por el componente ExampleValidator.
context.show(example_validator.outputs['anomalies'])

### Ingenierıa de caracterısticas

In [ ]:
# Definimos la ruta al archivo de módulo de constantes.
_cover_constants_module_file = 'cover_constants.py'

In [ ]:
%%writefile {_cover_constants_module_file}

# Definimos las claves de características que se escalarán utilizando el método MinMaxScaler.
SCALE_MINMAX_FEATURE_KEYS = [
        "Horizontal_Distance_To_Hydrology",
        "Vertical_Distance_To_Hydrology",
    ]

# Definimos las claves de características que se escalarán a un rango entre 0 y 1.
SCALE_01_FEATURE_KEYS = [
        "Hillshade_9am",
        "Hillshade_Noon",
        "Horizontal_Distance_To_Fire_Points",
    ]

# Definimos las claves de características que se escalarán utilizando el método StandardScaler.
SCALE_Z_FEATURE_KEYS = [
        "Elevation",
        "Slope",
        "Horizontal_Distance_To_Roadways",
    ]

# Definimos las claves de características cuyos valores son categóricos y se utilizará un vocabulario para su representación.
VOCAB_FEATURE_KEYS = ["Wilderness_Area"]

# Definimos las claves de características cuyos valores son representados como una cadena de caracteres y se utilizará un hash para su representación.
HASH_STRING_FEATURE_KEYS = ["Soil_Type"]

# Clave de la etiqueta de clasificación.
LABEL_KEY = "Cover_Type"

# Función de utilidad para renombrar la característica transformada.
def transformed_name(key):
    return key + '_xf'

In [ ]:
# Definimos el nombre del archivo del módulo de transformación.
_cover_transform_module_file = 'cover_transform.py'

In [ ]:
%%writefile {_cover_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import cover_constants

# Importamos constantes y funciones definidas en el módulo cover_constants.
_SCALE_MINMAX_FEATURE_KEYS = cover_constants.SCALE_MINMAX_FEATURE_KEYS
_SCALE_01_FEATURE_KEYS = cover_constants.SCALE_01_FEATURE_KEYS
_SCALE_Z_FEATURE_KEYS = cover_constants.SCALE_Z_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = cover_constants.VOCAB_FEATURE_KEYS
_HASH_STRING_FEATURE_KEYS = cover_constants.HASH_STRING_FEATURE_KEYS
_LABEL_KEY = cover_constants.LABEL_KEY
_transformed_name = cover_constants.transformed_name

# Definimos la función de preprocesamiento.
def preprocessing_fn(inputs):

    features_dict = {}

    # Escalamos las características utilizando la función scale_by_min_max.
    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature] 
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)
        
    # Escalamos las características a un rango entre 0 y 1.
    for feature in _SCALE_01_FEATURE_KEYS:
        data_col = inputs[feature] 
        features_dict[_transformed_name(feature)] = tft.scale_to_0_1(data_col)

    # Escalamos las características utilizando la puntuación Z.
    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature] 
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    # Aplicamos un vocabulario a las características categóricas.
    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature] 
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    # Convertimos las cadenas de caracteres en características numéricas mediante hashing.
    for feature in _HASH_STRING_FEATURE_KEYS:
        data_col = inputs[feature] 
        features_dict[_transformed_name(feature)] = tft.hash_strings(data_col, hash_buckets=10)

    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict

In [ ]:
# Creamos el componente Transform con los ejemplos de entrada y el esquema importado.
transform = tfx.components.Transform(examples=example_gen.outputs['examples'],
                                    schema=imported_schema.outputs['schema'],
                                    preprocessing_fn='cover_transform.preprocessing_fn') # We are using the .py file that was created instead of the above function

# Ejecutamos el componente Transform.
context.run(transform)

In [ ]:
# Obtenemos la URI de los ejemplos transformados generados por el componente Transform.
transformed_examples_uri = transform.outputs['transformed_examples'].get()[0].uri

# Listamos el contenido del directorio que contiene los ejemplos transformados.
os.listdir(transformed_examples_uri)

In [ ]:
# Mostramos el gráfico de transformación generado por el componente Transform.
context.show(transform.outputs['transform_graph'])

In [ ]:
# Obtenemos la URI del conjunto de datos transformado para el conjunto de entrenamiento.
uri_entrenamiento = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Obtenemos la lista de archivos TFRecord en la URI del conjunto de entrenamiento.
archivos_tfrecord = [os.path.join(uri_entrenamiento, nombre)
                     for nombre in os.listdir(uri_entrenamiento)]

# Creamos un conjunto de datos TFRecordDataset a partir de los archivos TFRecord transformados.
conjunto_transformado = tf.data.TFRecordDataset(archivos_tfrecord, compression_type="GZIP")

In [ ]:
def ob_reg(conjunto, num_registros):

    """
    Obtiene un número especificado de registros de un conjunto de datos TFRecord.

    Parámetros:
    - conjunto: Conjunto de datos TFRecord.
    - num_registros: Número de registros a obtener.

    Retorna:
    - registros: Lista de diccionarios que representan los registros.
    """
    
    registros = []
    
    for tfrecord in conjunto.take(num_registros):
        
        ejemplo_serializado = tfrecord.numpy()
        
        ejemplo = tf.train.Example()
        
        ejemplo.ParseFromString(ejemplo_serializado)
        
        diccionario_ejemplo = (MessageToDict(ejemplo))
        
        registros.append(diccionario_ejemplo)
        
    return registros

In [ ]:
registros_muestra_xf = ob_reg(conjunto_transformado, 1)

pp.pprint(registros_muestra_xf)

### Metadatos

In [ ]:
_util = 'util.py'

In [ ]:
%%writefile {_util}

import tensorflow as tf
import pandas as pd
from google.protobuf.json_format import MessageToDict

def get_records(dataset, num_records):
    
    '''Extrae registros del conjunto de datos dado.
    Args:
        dataset (TFRecordDataset): conjunto de datos guardado por ExampleGen
        num_records (int): número de registros para previsualizar
    '''    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

def display_types(types):
    # Función auxiliar para renderizar DataFrames para los tipos de artefactos y ejecuciones
    table = {'id': [], 'name': []}
    for a_type in types:
        table['id'].append(a_type.id)
        table['name'].append(a_type.name)
    return pd.DataFrame(data=table)

def display_artifacts(store, artifacts, base_dir):
    # Función auxiliar para renderizar DataFrames para los artefactos de entrada
    table = {'artifact id': [], 'type': [], 'uri': []}
    for a in artifacts:
        table['artifact id'].append(a.id)
        artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
        table['type'].append(artifact_type.name)
        table['uri'].append(a.uri.replace(base_dir, './'))
    return pd.DataFrame(data=table)

def display_properties(store, node):
    # Función auxiliar para renderizar DataFrames para las propiedades de artefactos y ejecuciones
    table = {'property': [], 'value': []}
    
    for k, v in node.properties.items():
        table['property'].append(k)
        table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
    
    for k, v in node.custom_properties.items():
        table['property'].append(k)
        table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
    
    return pd.DataFrame(data=table)

In [ ]:
import util as u_t
import ml_metadata as mlmd
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

# Configuramos la conexión a la base de datos SQLite
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = 'pipeline/metadata.sqlite'
connection_config.sqlite.connection_mode = 3

# Creamos el almacén de metadatos utilizando la configuración de conexión
store = metadata_store.MetadataStore(connection_config)

### Acceso a artefactos almacenados

In [ ]:
artifact_types = store.get_artifact_types()

In [ ]:
u_t.display_types(artifact_types)

In [ ]:
# Obtenemos los artefactos por tipo

list_artifact_type_schema = store.get_artifacts_by_type('Schema')
list_artifact_type_example = store.get_artifacts_by_type('Examples')
list_artifact_type_transform = store.get_artifacts_by_type('TransformGraph')
list_artifact_type_example_statistic = store.get_artifacts_by_type('ExampleStatistics')

In [ ]:
u_t.display_artifacts(store, list_artifact_type_schema, '.')

In [ ]:
u_t.display_artifacts(store, list_artifact_type_example, '.')

### Seguimiento de Artefactos

In [ ]:
def get_parent_artifacts(store, artifact):

    # Obtenemos el ID del artefacto de entrada
    artifact_id = artifact.id
    
    # Obtenemos los eventos asociados al ID del artefacto
    artifact_id_events = store.get_events_by_artifact_ids([artifact_id])
    
    # Del conjunto de `artifact_id_events`, obtenemos los IDs de ejecución de los eventos de SALIDA.
    # Convertimos a un conjunto para eliminar duplicados si los hay.
    execution_id = set( 
        event.execution_id
        for event in artifact_id_events # @REPLACE
        if event.type == metadata_store_pb2.Event.OUTPUT # @REPLACE
    )
    
    # Obtenemos los eventos asociados al ID de ejecución
    execution_id_events = store.get_events_by_execution_ids(execution_id)

    # Del conjunto de `execution_id_events`, obtenemos los IDs de artefacto de los eventos de ENTRADA.
    # Convertimos a un conjunto para eliminar duplicados si los hay.
    parent_artifact_ids = set( 
        event.artifact_id
        for event in execution_id_events
        if event.type == metadata_store_pb2.Event.INPUT
    )
    
    # Obtenemos la lista de artefactos asociados a los IDs de artefacto de entrada
    parent_artifact_list = [artifact for artifact in store.get_artifacts_by_id(parent_artifact_ids)]
    
    return parent_artifact_list

In [ ]:
# Obtenemos una instancia de artefacto del almacén de metadatos
artifact_instance = store.get_artifacts_by_type('TransformGraph')[0]

# Recuperamos los artefactos padres de la instancia
parent_artifacts = get_parent_artifacts(store, artifact_instance)

# Mostramos los resultados
u_t.display_artifacts(store, parent_artifacts, '.')